In [1]:
!pip -q install python-dotenv pinecone-client llama-index pymupdf


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
     ---------------------------------------- 0.1/8.4 MB 2.6 MB/s eta 0:00:04
     - -------------------------------------- 0.2/8.4 MB 2.9 MB/s eta 0:00:03
     - -------------------------------------- 0.3/8.4 MB 3.1 MB/s eta 0:00:03
     -- ------------------------------------- 0.6/8.4 MB 3.7 MB/s eta 0:00:03
     --- ------------------------------------ 0.7/8.4 MB 3.4 MB/s eta 0:00:03
     ---- ----------------------------------- 0.9/8.4 MB 3.5 MB/s eta 0:00:03
     ----- ---------------------------------- 1.1/8.4 MB 4.0 MB/s eta 0:00:02
     ------ --------------------------------- 1.4/8.4 MB 4.3 MB/s eta 0:00:02
     ------- -------------------------------- 1.7/8.4 MB 4.3 MB/s eta 0:00:02
     -------- ------------------------------- 1.8/8.4 MB 4.2 MB/s eta 0:00:02
     ---------- ----------------------------- 2.3/8.4 MB 4.5 MB/s eta 0


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# from google.colab import drive, files

# drive.mount("/content/drive/", force_remount=False)

# #Home directory:
# %cd "/content/drive/My Drive/Docs Ingestion/"

Mounted at /content/drive/
/content/drive/My Drive/Docs Ingestion


In [2]:
# sentence transformers
from llama_index.embeddings import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

ImportError: HuggingFaceEmbedding requires transformers to be installed.
Please install transformers with `pip install transformers`.

In [4]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 20.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.32-cp310-cp310-manylinux_2_35_x86_64.whl size=2403855 sha256=37db5732600aa031da284cee1b68b923e2f43118ee8bf5f2188ffaaf952fab90
  Stored in directory: /root/.cache/pip/wheels/fd/34/e2/592ae50a491d192807efe8a6abcb3622fb2190402252a7258f
Successfully built llama-cpp-python


In [5]:
from llama_index.llms import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

total size (MB): 7365.83


7025it [01:10, 100.07it/s]                          
AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'general.name': 'LLaMA v2', 'llama.embedding_length': '5120', 'llama.feed_forward_length': '13824', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.dimension_count': '128', 'llama.attention.head_count': '40', 'tokenizer.ggml.bos_token_id': '1', 'llama.block_count': '40', 'llama.attention.head_count_kv': '40', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'llama', 'general.file_type': '2'}


In [ ]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

CHROMADB

In [28]:
!pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 66.8 MB/s eta 0:00:00


#### Initialize Postgres

Using an existing postgres running at localhost, create the database we'll be using.

**NOTE**: Of course there are plenty of other open-source/self-hosted databases you can use! e.g. Chroma, Qdrant, Weaviate, and many more. Take a look at our [vector store guide](https://gpt-index.readthedocs.io/en/stable/core_modules/data_modules/storage/vector_stores.html).

**NOTE**: You will need to setup postgres on your local system. Here's an example of how to set it up on OSX: https://www.sqlshack.com/setting-up-a-postgresql-database-on-mac/.

**NOTE**: You will also need to install pgvector (https://github.com/pgvector/pgvector).

You can add a role like the following:
```
CREATE ROLE <user> WITH LOGIN PASSWORD '<password>';
ALTER ROLE <user> SUPERUSER;
```

In [29]:
import psycopg2

db_name = "vector_db"
host = "localhost"
password = "password"
port = "5432"
user = "root"
# conn = psycopg2.connect(connection_string)
conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}")

OperationalError: connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (::1), port 5432 failed: Cannot assign requested address
	Is the server running on that host and accepting TCP/IP connections?


In [19]:
import fitz

doc = fitz.open("./Docs/Fine-Tuning-pdf.pdf")
header = "Header"  # text in header
footer = "Page %i of %i"  # text in footer
for page in doc:
    page.insert_text((50, 50), header)  # insert header
    page.insert_text(  # insert footer 50 points above page bottom
        (50, page.rect.height - 50),
        footer % (page.number + 1, doc.page_count),
    )

docs = []

for page in doc:
  text = page.get_text()
  metadata = page.get_label()
  print(metadata)
  docs.append(text)

1
2
3
4
5
6
7
8
9
10
11


In [9]:
print(docs[0])

Applied
 Soft
 Computing
 46
 (2016)
 875–885
Contents
 lists
 available
 at
 ScienceDirect
Applied
 Soft
 Computing
j
 ourna
 l h
 o
 mepage:
 www.elsevier.com/locate/asoc
Fine-tuning
 Deep
 Belief
 Networks
 using
 Harmony
 Search
João
 Paulo
 Papa a,∗, Walter
 Scheirer b, David
 Daniel
 Cox b
a UNESP
 –
 Univ
 Estadual
 Paulista,
 Department
 of
 Computing,
 Bauru,
 Brazil
b Harvard
 University,
 Center
 for
 Brain
 Science,
 Cambridge,
 USA
a
 r
 t  i  c  l  e  
i  n
 f  o
Article
 history:
Received
 29
 May
 2015
Received
 in
 revised
 form
 15
 August
 2015
Accepted
 25
 August
 2015
Available
 online
 16
 September
 2015
Keywords:
Restricted
 Boltzmann
 Machines
Deep
 Belief
 Networks
Harmony
 Search
Meta-heuristics
a
 b
 s  t  r
 a  c  t
In this  paper,  we  deal  with  the  problem
 of Deep  Belief
 Networks
 (DBNs)  parameters
 ﬁne-tuning
 by
means
 of  a fast  meta-heuristic
 approach
 named
 Harmony
 Search
 (HS).  Although
 such  deep  learning-based
technique
 has  been  

In [15]:
text_parser = SentenceSplitter(
    chunk_size = 1024,
    separator = "."
)

In [16]:
text_chunks = []
doc_idxs = []
for doc_idx, page in enumerate(doc):
    page_text = page.get_text("text")
    cur_text_chunks = text_parser.split_text(page_text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [ ]:
print(text_chunks[0])

Costruzione manuale di Nodi con LLamaIndex in modo da contenere i metadata

In [18]:
# from llama_index.schema import TextNode

# nodes = []
# for idx, text_chunk in enumerate(text_chunks):
#     node = TextNode(
#         text=text_chunk,
#     )
#     src_doc = doc[doc_idxs[idx]]
#     node.metadata = src_doc.metadata
#     nodes.append(node)

AttributeError: 'Page' object has no attribute 'metadata'

In [21]:
from llama_index.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    # src_doc = doc[doc_idxs[idx]]
    # node.metadata = src_doc.metadata
    nodes.append(node)

In [22]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding